In [86]:
import nupack as nu 
import numpy as np 
from seqwalk import design
import math
import sys
import random 

from orthogonal import *








## testing toeholds.py

In [54]:
# Force fresh reload by removing from cache
if 'toeholds' in sys.modules:
    del sys.modules['toeholds']

from toeholds import *
random.seed(69420)

In [55]:
n=4000
random_dna_seq = "".join(random.choices("ACGT", k=n))

In [56]:
oligo_length = 150 
barcode_length = 18 
fragment_length = oligo_length - 2 * barcode_length
max_final_c_domain_length = oligo_length - barcode_length
toehold_length = 10
toehold_search_range = 15
identity_threshold = 0.6
args = oligo_length, barcode_length, fragment_length, max_final_c_domain_length, toehold_length, toehold_search_range, identity_threshold

In [57]:
putative_toeholds = generate_putative_toeholds(random_dna_seq, args)


In [58]:
combinations = toehold_combinations(putative_toeholds, args, int(1e2))

In [59]:
model =nu.Model(material="dna", celsius=50)
ncores = mp.cpu_count()
conc = 1e-8
data = evaluate_combinations(combinations, model, ncores, conc)

100%|██████████| 100/100 [05:43<00:00,  3.44s/it]


In [60]:
data

array([(2.0, 0.0002982131917963483, 0.0034195899691726755, 0.29719478074137945, 0.39297144384228755, 0.0),
       (2.0, 0.00013790091111591475, 0.18107369006266782, 0.2825317398358466, 0.4798788657255263, 0.0),
       (2.0, 0.00012987653077949088, 0.18691148749352196, 0.29053538101426357, 0.4165292536038077, 0.0),
       (2.0, 5.7554774804939986e-05, 0.12244369565369033, 0.29053538101426357, 0.39654303907422606, 0.0),
       (2.0, 4.3340717759379406e-05, 0.00131557061513495, 0.3169804155722641, 0.4459487239256276, 0.0),
       (2.0, 4.3340717759379406e-05, 0.032325394981389685, 0.23883336024057797, 0.2522550570626099, 0.0),
       (2.0, 6.621867812904192e-05, 0.017245275943643584, 0.24328716211957407, 0.4598605894067417, 0.0),
       (2.0, 0.0001930517487296648, 0.017245275943643584, 0.3169804155722641, 0.4483133453002094, 0.0),
       (2.0, 7.344049818033725e-05, 0.006044424397208339, 0.36260353985630733, 0.39311289471762645, 0.0),
       (3.0, 0.0002543048765568198, 0.002948761392883

In [61]:
bestid = rank_combinations_weighted(data)
toeholds = combinations[bestid]

36
edit_distance_ranks
1
ontarget_prob_ranks
18
offtarget_prob_ranks
4
on_target_defect_ranks
4
off_target_defect_ranks
21
tmdelta_ranks
1
raw data
(3.0, 0.00017141230034021178, 0.00037987984944679585, 0.19872595746426908, 0.43801074399227014, 0.0)


In [62]:
combination = combinations[1]
library = list(zip(*combination))[0]
nu_mat, on_t = nupack_matrix_no_mp(library, model, conc, 1)
on_t

array([7.89079791e-04, 7.76643104e-03, 2.60649550e-02, 1.98125140e-03,
       1.43557396e-02, 3.06697288e-03, 9.07445679e-04, 1.42622627e-03,
       2.32535450e-03, 1.17301302e-03, 5.94545994e-04, 3.45658644e-03,
       6.47793776e-03, 5.17136213e-03, 1.37900911e-04, 2.22268428e-03,
       2.03797413e-03, 1.24418776e-03, 8.45387569e-04, 2.02543908e-01,
       1.13889114e-01, 3.48989594e-04, 5.36253038e-02, 4.77750332e-03,
       1.12963549e-03, 7.28139804e-01, 7.26973349e-03, 1.05848362e-02,
       6.85495267e-03, 1.61845186e-02, 3.01848805e-02, 4.56691021e-04,
       7.17970444e-03, 2.33510654e-02, 4.50903948e-02, 3.77464118e-03,
       1.23279450e-03, 4.60518801e-02])

## testing union

In [63]:
# note to self
# barcodes and toeholds are not joined in testing due to 
# the nature of sidewinder

In [96]:
# Force fresh reload by removing from cache
if 'union' in sys.modules:
    del sys.modules['union']

from union import *

In [97]:
toeholds_seqs = [toeholds[i][0] for i in range(len(toeholds))]

In [100]:
barcode_sets = generate_putative_barcodes(toeholds_seqs, 5,9,barcode_length,100)

/Users/acheron/miniforge3/envs/mamba/lib/python3.9/site-packages/seqwalk/design.py:28: UserWarning: Falling back to Hierholzer algorithm for odd-k, ACGT, RC free
  warn("Falling back to Hierholzer algorithm for odd-k, ACGT, RC free")


123
38


In [101]:
len(barcode_sets)

100

In [102]:
barcode_data = evaluate_barcodes(barcode_sets, toeholds_seqs, model, ncores, conc)

100%|██████████| 100/100 [16:30<00:00,  9.90s/it] 


In [105]:
barcode_data 

array([(3.0, 0.18544049992429626, 0.5452440404866327, 0.45959935822761483, 0.4031511119678402, 0.0),
       (3.0, 0.18544049992429626, 0.5452440404866327, 0.6355796516164355, 0.4031511119678402, 0.0),
       (3.0, 0.13475204367228494, 0.04678540406582884, 0.42792309697772396, 0.4883801099967624, 0.0),
       (3.0, 0.13475204367228494, 0.5214185355262884, 0.6355796516164355, 0.41316048409013106, 0.0),
       (3.0, 0.27668201749637605, 0.27931995001719556, 0.6355796516164355, 0.41316048409013106, 0.0),
       (3.0, 0.6110690447312901, 0.12045635464400196, 0.554550879516882, 0.4511580404872655, 0.0),
       (3.0, 0.27668201749637605, 0.5452440404866327, 0.6355796516164355, 0.4031511119678402, 0.0),
       (3.0, 0.13475204367228494, 0.5452440404866327, 0.5417682462810005, 0.4031511119678402, 0.0),
       (3.0, 0.18544049992429626, 0.12045635464400196, 0.6355796516164355, 0.41316048409013106, 0.0),
       (3.0, 0.6110690447312901, 0.030250891016833804, 0.5417682462810005, 0.4131604840901310

In [122]:
best_barcodeid = rank_combinations_weighted(barcode_data)

99
edit_distance_ranks
1
ontarget_prob_ranks
8
offtarget_prob_ranks
2
on_target_defect_ranks
2
off_target_defect_ranks
5
tmdelta_ranks
1
raw data
(3.0, 0.6110690447312901, 0.015967127717091113, 0.4052123171084145, 0.6013874680867247, 0.0)


In [107]:
bb=barcode_sets[best_barcodeid]

In [88]:
np_probs, on_probs = nupack_matrix_mp(bb, model, 1e-8, 10, 1)

np.argmin(on_probs)

Generating...



100%|██████████| 38/38 [00:01<00:00, 25.67it/s]


16

In [123]:
am = alignment_matrix(bb, 1)
np.fill_diagonal(am, np.inf)

i = np.argmin(am)


In [124]:
r = i // len(bb)
c = i - r * len(bb)

In [125]:
am[r,c]

5.0

In [126]:
bb[r]

'GCCTCACGTACCGCCTAG'

In [127]:
bb[c]

'CTGGGCGGTAGTAAGGAA'

In [89]:
nu.reverse_complement(bb[16])

'ACGTTACTACGTAACTAC'

In [90]:
bb[16]

'GTAGTTACGTAGTAACGT'

In [91]:
np_ontarget(bb[16], model, 1e-8, 1)/(1e-8)

0.07710075390555396

In [47]:
domains =get_domains(toeholds, random_dna_seq, toehold_length)
domains 

array(['GCGTGAGTAGCCATCCTGCGGAAGATGTGAGAGCGGAGTCAGTGGGTGGAGCGATACTAAATGTCAAGCGTATACCATCTCCAAATCGAACAAAATATCCGACAGAACTT',
       'ACTGATGTTCACCCGGAGCTATGAATGCTTCTCATTATTGCTCGCACCAGATTACAATGTTCTTTCAGCGTTGGACCGTTCGTGGTATTGTT',
       'GTTCTTAACAGTGCCCGCAGGAATCATCCCGGCGCGCGTGGCCAATTCCACGGGAGAGTTCGTCTAACTATAGATTAAAATCAATATACCCAATCC',
       'GTCTGATTTAATCAGGGGTAGAGTGTGCAGAATGTAACCGTGCAATACGGAGCACACTGGAAGTCCCTGTCCGCCTTTTTGGTGACGGTTGATGTCC',
       'ACCGTGCATAACTATGCCTTGTGACCCGTTATCTCCGTCATATCTGACGTACGGCCACAAGTACGTATAACCCCGTTGTCATCGGCTACGGGGC',
       'CTTTGCAACGCAATTTCTGCATCGTTCAGGCTACACGCTTGACAAGATTGTTAGGACTCATGTGCCTCCCCTAAGGTAACGGCTTCGAA',
       'TTTGTCGGGATACTAGACCTTTTGAACTTGCAGACGAACCACTGATGTATCATTAATGAGTCCTTATCCAAAACTGAAGGAGCGATTAGCCCTACTG',
       'TAGTACCATGGAAAGAGTATGGTGTATATAACCTTTCCACACTGTAAGCCGGCACCGCCTGGCGGGGAGTAACGCCGGGGAAGTGCTGATACGTC',
       'TCCCGCTGACTAAATCGGGGGGAGAAATAGCATCCCATGCGGCCCAAAGTCCTTCTAGCGAGACACGTTCGAGTACTTGTTGCTGTACCTGTTG',
       'GTACGACAACTCAAAGGATCAGTTATTCC

## misc tests

In [144]:
# test toehold and domain homogeneity 
test_seq = ""
for i in range(len(toeholds)):
    test_seq += domains[i]
    test_seq += toeholds[i][0]

test_seq += domains[len(domains)-1]

assert(test_seq == random_dna_seq)

In [145]:
# i wonder if the ordering of best/worst barcodes is important...

In [119]:
toeholds[2][1] - toeholds[1][1] + toehold_length

123

In [120]:
fragment_length 

114

In [160]:
max = 2000
for i in range(1,len(toeholds)-2):
    if toeholds[i+1][1] - toeholds[i][1] + toehold_length < max:
        max = toeholds[i+1][1] - toeholds[i][1] + toehold_length
    if max == 94:
        print(i)
max

5
6
7
8
9
10
11
12
13
14
15
16
17


94

In [161]:
toeholds[5]

('GTTAGGACTC', 588)

In [162]:
toeholds[6]

('CGAACCACTG', 672)

In [163]:
672-588+toehold_length

94

In [185]:
len(random_dna_seq[putative_toeholds[1][-1][1]: (putative_toeholds[0][1][1] + 1 + toehold_length)])

0

In [196]:
len(random_dna_seq[putative_toeholds[0][0][1]:(putative_toeholds[1][-1][1] + toehold_length)])

117

In [191]:
putative_toeholds[0][0][0]

'GAACTTTCAT'

In [195]:
putative_toeholds[1][-1][0]

'TCGAGTAAAA'

In [180]:
putative_toeholds[1][0][1] - putative_toeholds[0][-1][1] + 1 + toehold_length

90

In [184]:
random_dna_seq[0:2]

'GC'

In [183]:
random_dna_seq

'GCGTGAGTAGCCATCCTGCGGAAGATGTGAGAGCGGAGTCAGTGGGTGGAGCGATACTAAATGTCAAGCGTATACCATCTCCAAATCGAACAAAATATCCGACAGAACTTTCATGACATGACTGATGTTCACCCGGAGCTATGAATGCTTCTCATTATTGCTCGCACCAGATTACAATGTTCTTTCAGCGTTGGACCGTTCGTGGTATTGTTCGAGTAAAATGTTCTTAACAGTGCCCGCAGGAATCATCCCGGCGCGCGTGGCCAATTCCACGGGAGAGTTCGTCTAACTATAGATTAAAATCAATATACCCAATCCGTCGGTTACGGTCTGATTTAATCAGGGGTAGAGTGTGCAGAATGTAACCGTGCAATACGGAGCACACTGGAAGTCCCTGTCCGCCTTTTTGGTGACGGTTGATGTCCATAGATGTGCACCGTGCATAACTATGCCTTGTGACCCGTTATCTCCGTCATATCTGACGTACGGCCACAAGTACGTATAACCCCGTTGTCATCGGCTACGGGGCCCTATTCAGCCTTTGCAACGCAATTTCTGCATCGTTCAGGCTACACGCTTGACAAGATTGTTAGGACTCATGTGCCTCCCCTAAGGTAACGGCTTCGAAATCCAACCGATTTGTCGGGATACTAGACCTTTTGAACTTGCAGACGAACCACTGATGTATCATTAATGAGTCCTTATCCAAAACTGAAGGAGCGATTAGCCCTACTGTTTTAATGCATAGTACCATGGAAAGAGTATGGTGTATATAACCTTTCCACACTGTAAGCCGGCACCGCCTGGCGGGGAGTAACGCCGGGGAAGTGCTGATACGTCTAACTTGGTGTCCCGCTGACTAAATCGGGGGGAGAAATAGCATCCCATGCGGCCCAAAGTCCTTCTAGCGAGACACGTTCGAGTACTTGTTGCTGTACCTGTTGTGACTCGTGCGTACGACAACTCAAAGGATCAGTTATTCCTACATGCATCGATTCA

In [197]:
fragment_length 

114

In [51]:
from orthogonal import *
s1 = toeholds_seqs[0]
s2 = toeholds_seqs[1]

ensemble_defect_pair(s1, s2, model, ss_structure(len(s1)*2))

0.1540953284011465

In [53]:
ss_structure(len(s1)*2)

'....................'